In [1]:
import numpy as np


# OPTIONS
skip_pools = 0 # set to true to use only bracket data


# EXTRACT DATA FROM TABLES
''' First get number of datapoints and number of player names '''
    
file = open( 'smashdata.csv' )
numitems = 0
playernames = {}
charnames = {}
game_outcomes = []
tournaments = {}
tourney_years = {}
for line in file:
    line = line.strip()
    parts = line.split(',')
    if parts[7] == 'pools':
        if skip_pools == 1:
            continue
    tourney_years[ parts[0] ] = 1
    tournaments[ parts[1] ] = 1
    numitems = numitems + 1
    playernames[ parts[2] ] = 1
    playernames[ parts[3] ] = 1
    charnames[ parts[4] ] = 1
    charnames[ parts[5] ] = 1
    game_outcomes.append(parts[6])


print("Smash 64 Results from major tournaments")
print("\n\tData from following tournaments:")
for key in tournaments:
    print('\t\t' + key)
print("\nDuring these years:")
for key in tourney_years:
    print('\t\t' + key)
    
numchars = len(charnames)
numnames = len(playernames)
if skip_pools == 1:
    print("\tIncludes bracket games only")
if skip_pools != 1:
    print("\tIncludes bracket and pools")
print("\tData taken from all recorded games.")
print( '\nNumber of players in data: ' + str(numnames) )
print( 'Number of games in data: ' + str(numitems) )

Smash 64 Results from major tournaments

	Data from following tournaments:
		snosa2
		pound
		genesis 3
		beast7
		goml

During these years:
		2017
		2016
	Includes bracket and pools
	Data taken from all recorded games.

Number of players in data: 128
Number of games in data: 554


In [2]:
# Next retrieve character-based results

''' Next construct hastables to map from name to number '''
number2name = {}
name2number = {}
whichname = 0
for item in playernames:
    name2number[ item ] = whichname
    number2name[ whichname ] = item
    whichname = whichname + 1


''' Now construct hashtables for characters '''
number2char = {}
char2number = {}
whichchar = 0
for item in charnames:
    char2number[ item ] = whichchar
    number2char[ whichchar ] = item
    whichchar = whichchar + 1

''' Construct adjacency matrices for game stats ??? '''
# GameMat gives number of games played between players.
# CharGameMat[x,y] gives number of games played between characters x and y
# CharOutcomeMat[x,y] gives number of wins of char y over char x
GameMat = np.zeros((numnames, numnames))
CharGameMat = np.zeros((numchars, numchars))
CharOutcomeMat = np.zeros((numchars, numchars))
pool_or_bracket = []

file.close()
file = open( 'smashdata.csv' )
line_number = 0

char_wins = np.zeros((numchars,1))
for line in file:
    line = line.strip()
    parts = line.split(',')
    pool_or_bracket.append(parts[-1])
    if parts[7] == 'pools':
        if skip_pools == 1:
            continue
    xj = name2number[ parts[2] ]
    yj = name2number[ parts[3] ]
    GameMat[ xj, yj ] = GameMat[ xj, yj ] + 1
    GameMat[ yj, xj ] = GameMat[ yj, xj ] + 1
    xchar = char2number[ parts[4] ]
    ychar = char2number[ parts[5] ]
    winchar = xchar
    losechar = ychar
    if int(parts[6]) == 2:
        winchar = ychar
        losechar = xchar
    #if int(parts[6]) == 1:
    CharOutcomeMat[ losechar, winchar ] = CharOutcomeMat[ losechar, winchar ] + 1
    if winchar != losechar:
        char_wins[ winchar, 0 ] = char_wins[ winchar, 0 ] + 1
    
    CharGameMat[ xchar, ychar ] = CharGameMat[ xchar, ychar ] + 1
    CharGameMat[ ychar, xchar ] = CharGameMat[ ychar, xchar ] + 1
    line_number = line_number + 1

'''
These data structures contain actual tournament game/match data:
    GameMat
    CharGameMat
    
These contain maps from data structures indices to player/character names:
    char2number
    number2char
    name2number
    number2name
    
'''
char1 = []
char2 = []
char_tots = []
outcome_prct = []
num_matches = []
num_outcomes = []
for i in range( len(char2number) ) :
    for j in range( i, len(char2number) ) :
        char1.append(number2char[i])
        char2.append(number2char[j])
        num_matches.append(CharGameMat[ i,j ])
        num_outcomes.append(CharOutcomeMat[ j,i ])
        if CharGameMat[i,j] != 0:
            outcome_prct.append(CharOutcomeMat[j,i]/CharGameMat[ i,j ])
        if CharGameMat[i,j] == 0:
            outcome_prct.append(-5)

for i in range(len(char2number)):
    char_tots.append( [number2char[i],0] )
    for j in range( len(char2number) ):
        char_tots[i][1] = char_tots[i][1] + CharGameMat[ i,j ]

        
# NOW PRINT
print("Matchup percentages \n")
print("char1\tchar2 \t# games \tchar1 wins \tprct win by char 1")
inds = list(reversed(np.argsort(num_matches)))
for i in inds:
    if num_matches[i] < 1:
        continue
    print( char1[i] + '\t' + char2[i] + '\t' + str(num_matches[ i ] ) + '\t\t' + str(num_outcomes[i]) + '\t\t' + ("%.2f" % outcome_prct[i]) )

char_tots
charnames = []
charnums = []
for j in range(len(char_tots)):
    charnames.append(char_tots[j][0])
    charnums.append(char_tots[j][1])

inds = list(reversed(np.argsort(charnums)))
sorted_data = []
for j in inds:
    sorted_data.append(char_tots[j])

# This time without dittos
char_tots
charnames = []
charnums = []
charwins = char_wins
for j in range(len(char_tots)):
    charnames.append(char_tots[j][0])
    charnums.append(char_tots[j][1] - CharGameMat[j,j])

inds = list(reversed(np.argsort(charnums)))

print("\nCharacter-based wins in recorded games\n(excludes dittos)")
print("char\t\t# games\t\t% wins")
for i in inds:
    #    print( charnames[i] + '\t' + str(charnums[ i ] ) + '\t' + str(charwins[i]) + ("%.2f" % (charwins[i][0]/charnums[i]) ) )
    print( charnames[i] + '\t\t' + str(charnums[ i ] ) + '\t\t' + ("%.2f" % (charwins[i][0]/charnums[i]) ) )

# Print just character appearances
# (including dittos)
charappearances = sum(CharGameMat)
inds = list(reversed(np.argsort(charappearances)))
print("\nTotal games " + str(sum(sum(CharGameMat))/2 ) + " (two characters appear per game)")
print("char\t\t# appearances" + '\t' + "% of total")
for i in inds:
    #print( charnames[i] + '\t\t' + str(sum(CharGameMat[ i, : ])) + '\t\t' + ("%.2f" % (sum(CharGameMat[ i, : ])/(sum(sum(CharGameMat))) ) ) )
    print( charnames[i] + '\t\t' + str(charappearances[ i ]) + '\t\t' + ("%.2f" % (charappearances[i]/(sum(sum(CharGameMat))) ) ) )

Matchup percentages 

char1	char2 	# games 	char1 wins 	prct win by char 1
pika	pika	120.0		60.0		0.50
falcon	pika	74.0		36.0		0.49
pika	kirby	51.0		30.0		0.59
pika	yoshi	49.0		25.0		0.51
falcon	falcon	34.0		17.0		0.50
fox	pika	25.0		6.0		0.24
pika	puff	22.0		12.0		0.55
falcon	yoshi	22.0		8.0		0.36
falcon	fox	20.0		10.0		0.50
fox	fox	18.0		9.0		0.50
falcon	kirby	17.0		4.0		0.24
yoshi	kirby	16.0		8.0		0.50
link	kirby	16.0		9.0		0.56
pika	mario	12.0		6.0		0.50
puff	kirby	11.0		4.0		0.36
falcon	puff	11.0		8.0		0.73
falcon	link	9.0		5.0		0.56
ness	falcon	8.0		2.0		0.25
fox	kirby	8.0		6.0		0.75
yoshi	puff	8.0		6.0		0.75
fox	yoshi	7.0		5.0		0.71
falcon	samus	7.0		4.0		0.57
yoshi	mario	7.0		1.0		0.14
kirby	kirby	6.0		3.0		0.50
fox	puff	6.0		3.0		0.50
dk	dk	6.0		3.0		0.50
puff	mario	6.0		5.0		0.83
pika	link	5.0		2.0		0.40
luigi	mario	4.0		1.0		0.25
ness	yoshi	4.0		1.0		0.25
yoshi	yoshi	4.0		2.0		0.50
pika	dk	3.0		3.0		1.00
yoshi	link	3.0		2.0		0.67
kirby	mario	3.0		2.0		0.67
ness	pika	3.0		1.0

In [3]:
# GET PLAYER DATA
''' Get player game totals, and char totals '''
    
file.close()
file = open( 'smashdata.csv' )
numitems = 0
playerchargames = {}
playercharwins = {}
for line in file:
    line = line.strip()
    parts = line.split(',')
    if parts[-1] == 'pools':
        if skip_pools == 1:
            continue
    gameplayernames = [ parts[2], parts[3] ]
    gamecharnames = [ parts[4], parts[5] ]
    for j in [0,1]:
        current_playerchar = gameplayernames[j] + '-' + gamecharnames[j]
        playerchargames[ current_playerchar ] = 0
        playercharwins[ current_playerchar ] = 0
file.close()


file = open( 'smashdata.csv' )
for line in file:
    line = line.strip()
    parts = line.split(',')
    if parts[-1] == 'pools':
        if skip_pools == 1:
            continue
    gameplayernames = [ parts[2], parts[3] ]
    gamecharnames = [ parts[4], parts[5] ]
    for j in [0,1]:
        current_playerchar = gameplayernames[j] + '-' + gamecharnames[j]
        playerchargames[ current_playerchar ] = playerchargames[ current_playerchar ] + 1
    whichwin = int(parts[6]) - 1
    winningplayer = gameplayernames[whichwin] + '-' + gamecharnames[whichwin]
    playercharwins[winningplayer] = playercharwins[winningplayer] + 1
file.close()

# Re-organize to sort by wins
playchar2num = {}
num2playchar = []
num2wins = []
numitems = 0
for key, val in playerchargames.items():
    playchar2num[key] = numitems
    num2playchar.append(key)
    num2wins.append(val)
    numitems = numitems + 1
    
inds = list(reversed(np.argsort(num2wins)))
print("\nPlayer-character game data")
print("\nplayer-character" + '\t\t' + "# games" + '\t\t' + "# wins" + '\t\t' + "% win")
for j in range(len(playchar2num)):
    idx = inds[j]
    key = num2playchar[idx]
    val = int(playerchargames[key])
    valwins = int(playercharwins[key])
    print( key.ljust(20) + '\t\t' + str(val) + '\t\t' + str(valwins) + '\t\t' +  ("%.2f" % (valwins/val) ) )

    


Player-character game data

player-character		# games		# wins		% win
revan-kirby         		53		31		0.58
barksanchez-pika    		53		27		0.51
kerokeroppi-pika    		46		25		0.54
tacos-yoshi         		31		21		0.68
mariguas-pika       		29		19		0.66
superboomfan-falcon 		29		22		0.76
fireblaster-yoshi   		26		11		0.42
wizzrobe-yoshi      		24		10		0.42
shears-pika         		23		11		0.48
thez-pika           		23		13		0.57
superboomfan-kirby  		21		20		0.95
wangera-puff        		20		15		0.75
superboomfan-pika   		18		15		0.83
derek-pika          		17		10		0.59
wario-pika          		17		13		0.76
isai-link           		16		11		0.69
ld-fox              		15		10		0.67
tacos-pika          		14		7		0.50
dexter-pika         		14		3		0.21
tacos-falcon        		12		6		0.50
kimimaru-mario      		12		5		0.42
jaimehr-falcon      		12		6		0.50
jam-pika            		12		7		0.58
clubbadubba-pika    		11		6		0.55
isai-fox            		11		7		0.64
isai-mario          		10		7		0.70
ctg-puff            		10		3		

In [16]:

PLAYER_NAME = 'thez'

print("Smash 64 Tournament Results")
print("\n\tData from following tournaments:")
for key in tournaments:
    print('\t\t' + key)

numchars = len(charnames)
numnames = len(playernames)
if skip_pools == 1:
    print("\tIncludes bracket games only")
if skip_pools != 1:
    print("\tIncludes bracket and pools")
    
print("\nplayer-character" + '\t\t' + "# games" + '\t\t' + "# wins" + '\t\t' + "% win")
inds = list(reversed(np.argsort(num2wins)))
total_games = 0
total_wins = 0
for j in range(len(playchar2num)):
    idx = inds[j]
    key = num2playchar[idx]
    ply_nm_len = len(PLAYER_NAME)
    if key[0:ply_nm_len] != PLAYER_NAME:
        continue
    val = int(playerchargames[key])
    valwins = int(playercharwins[key])
    total_games = total_games + val
    total_wins = total_wins + valwins
    print( key.ljust(20) + '\t\t' + str(val) + '\t\t' + str(valwins) + '\t\t' +  ("%.2f" % (valwins/val) ) )
print( '\n' + "TOTALS".ljust(20) + '\t\t' + str(total_games) + '\t\t' + str(total_wins) + '\t\t' +  ("%.2f" % (total_wins/total_games) ) )

Smash 64 Tournament Results

	Data from following tournaments:
		snosa2
		pound
		genesis 3
		beast7
		goml
	Includes bracket and pools

player-character		# games		# wins		% win
thez-pika           		23		13		0.57
thez-fox            		3		2		0.67
thez-yoshi          		1		0		0.00
thez-falcon         		1		0		0.00

TOTALS              		28		15		0.54
